In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gp
from os.path import join, basename
from datetime import date, datetime
import os

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../3-postprocess/'))
import xstats as xs 

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from plot_tools import *
import cartopy.crs as ccrs
import seaborn as sns

In [ ]:
root = r'/scratch/compound_hotspots'
ddir = join(root, 'data', '4-postprocessed')
fdir = join(root, 'reports', 'figures')

In [ ]:
rc = {'savefig.bbox': 'tight',  'savefig.format': 'png', 'savefig.dpi':300}
context = 'paper'
# sns.set(context=context, style='whitegrid', font_scale=0.75 if context == 'talk' else 1., rc=rc)
sns.set(context=context, style='whitegrid', font_scale=1.2 if context == 'paper' else 1., rc=rc)
s=30
crs = ccrs.Robinson()
crs_sub = ccrs.PlateCarree()
cmap_div = sns.diverging_palette(220, 10, s=75, l=40, sep=1, as_cmap=True)
cmap_turbo_r = ListedColormap([
    interpolate_cmap(google_turbo_data, x) for x in np.linspace(1, 0, 256)
])

bmap_kwargs = dict()
plot_kwargs=dict(edgecolor=(0.5, 0.5, 0.5, 0.8), linewidth=0.5, legend=False, zorder=2)
box_kwargs=dict(whis=[5,95], boxprops=dict(linewidth=1.5), medianprops=dict(linewidth=1.5), 
                showfliers=True, flierprops=dict(markersize=2))

scen_cmap = {'surge': 'black', 'seas': 'blue', 'tide': 'cyan'}
scen_mmap  = {'surge': 'o', 'seas': '^', 'tide': 'd'}

regions = {
#     'A': (-90.0, 41.0, -50.0, 65.0), # 40 x 24
    'A': (-12.0, 47.0, 28.0, 65.0), # 30 x 18
#     'C': (70.0, -10.0, 130.0, 26.0),  # 60 x 36
}
from string import ascii_uppercase as letters

locs = [1230, 1648, 1618] #, 3417, 2891, 990]
riv_names = {
    990: 'Penobscot',
    1648: 'Rhine',
    1230: 'Amazon',
    1618: 'Volta',
    3417: 'Han',
    2891: 'Lenya',
    1617: 'Seine',
    1021: ''
}

driver_names = {
    'Htide': 'tide',
    'Hskewsurge': 'surge',
    'Hsurge': 'surge',
    'Q': 'discharge',
    'Qmsl': 'discharge',
    'Htot': 'Htot'

}

In [ ]:
attrs_fn = join(ddir, 'rivmth_mean_attrs.csv')
attrs = pd.read_csv(attrs_fn, index_col='index').rename(columns={'rivmth_lat':'lat', 'rivmth_lon':'lon'}).drop(3354)
attrs['Q_mean_log10'] = np.log10(np.maximum(attrs['Q_mean'].values, 0.001))
attrs['markersize'] = np.maximum((attrs['Q_mean_log10'] / attrs['Q_mean_log10'].max()),0) * 50 + 10

attrs['Q_amax_log10'] = np.log10(np.maximum(attrs['Q_amax'].values, 0.001))
attrs['uparea_log10'] = np.log10(np.maximum(attrs['uparea'].values, 0.001))
attrs['Hseasrange'] = attrs['Hseas_amax']-attrs['Hseas_amin']
attrs['mean_drain_length'] = attrs['mean_drain_length']/1e3 #[km]
attrs['mean_drain_slope'] = attrs['mean_drain_slope']*1e3 #[m/km]

## drivers of AM water levels

In [ ]:
from peaks import peaks_over_threshold, get_peaks
import xstats as xs

In [ ]:
wdw=1

# read data
fn_in = join(ddir, 'rivmth_reanalysis.zarr')
fn_out = join(ddir, f'rivmth_h_am_wdw{wdw}.nc')
scenarios = ['surge', 'msl']
vars_ = ['h', 'Htide', 'Hskewsurge', 'Hsurge', 'Htot', 'Qmsl', 'Q']
rm = {'Hskewsurge_day':'Hskewsurge', 'Htot_day_max': 'Htot', 'Htide_day_max': 'Htide', 'Hsurge_day_max': 'Hsurge'}
ds = xr.open_zarr(fn_in).sel(scen=scenarios).drop(['Hsurge', 'Htide']).rename(rm)
ds['Qmsl'] = ds['Q'].sel(scen='msl')
ds['h'] = ds['WSE']-ds['z0']
ds = ds[vars_].sel(scen='surge').drop('scen')#.isel(ensemble=0).sel(index=locs)

# window max driver
ds_wdwmax = ds.rolling(time=wdw*2+1, min_periods=1, center=True).construct('window').max('window').astype(np.float32)

# get AM h peaks
peaks  = get_peaks(ds['h'], min_dist=30, dim='time').reset_coords(drop=True).reindex_like(ds).compute()
hpeaks = peaks.where(np.isfinite(peaks), -np.inf)
hpeaks_yr = hpeaks.groupby('time.year')
hpeaks_am = hpeaks_yr == hpeaks_yr.max('time')
hpeaks_doy = hpeaks['time'].dt.dayofyear

# combine h AM peaks, drivers and return periods and keep only an. max.
ds_am_h = xr.merge([
    hpeaks_doy,
    ds_wdwmax,
]).where(hpeaks_am, -np.inf).groupby('time.year').max('time').compute()
ds_am_h.transpose('year', 'ensemble', 'index').to_netcdf(fn_out)

## timing

In [ ]:
wdw=1

# read data
fn_in = join(ddir, 'rivmth_reanalysis.zarr')
fn_out2 = join(ddir, f'rivmth_am_events_wdw{wdw}.nc')
scenarios = ['surge', 'msl']
vars_ = ['h', 'Htide', 'Hskewsurge', 'Hsurge', 'Htot', 'Qmsl', 'Q']
rm = {'Hskewsurge_day':'Hskewsurge', 'Htot_day_max': 'Htot', 'Htide_day_max': 'Htide', 'Hsurge_day_max': 'Hsurge'}
ds = xr.open_zarr(fn_in).sel(scen=scenarios).drop(['Hsurge', 'Htide']).rename(rm)
ds['Qmsl'] = ds['Q'].sel(scen='msl')
ds['h'] = ds['WSE']-ds['z0']
ds = ds[vars_].sel(scen='surge').drop('scen')#.isel(ensemble=0).sel(index=locs)

# window max driver
ds_wdwmax = ds.rolling(time=wdw*2+1, min_periods=1, center=True).construct('window').max('window').astype(np.float32)

# get driver AM events 
ds_lst = []
for v in vars_:
    min_dist = 3 if v.startswith('H') else 14
    peaks  = get_peaks(ds[v], min_dist=min_dist, dim='time').reset_coords(drop=True).reindex_like(ds).compute()
    peaks = peaks.where(np.isfinite(peaks), -np.inf)
    peaks_yr = peaks.groupby('time.year')
    peaks_am = peaks_yr == peaks_yr.max('time')
    peaks_doy = peaks['time'].dt.dayofyear
    peaks_doy.name = f'{v}_doy'
    peaks.name = f'{v}_am'
    _da = [peaks, peaks_doy]
    if v != 'h':
        _da.append(ds_wdwmax[['h']].rename({'h': f'h_{v}_am'}))
    _ds = xr.merge(_da).where(peaks_am, -np.inf).groupby('time.year').max('time')
    ds_lst.append(_ds)
xr.merge(ds_lst).to_netcdf(fn_out2)

In [ ]:
wdw=1
fn_out2 = join(ddir, f'rivmth_h_am_wdw{wdw}.nc')

# read data
fn_am = join(ddir, f'rivmth_am_events_wdw{wdw}.nc')
ds_am = xr.open_dataset(fn_am).sel(ensemble=['anu'])
# ds_am

In [ ]:
from temp_stats import mean_flood_day_diff, corr_flood_day, circ_var, rayleightest, doy_to_circ, circ_mean, circ_to_doy

drivers = ['Htide','Hskewsurge', 'Q']

wse_doy_p = rayleightest(doy_to_circ(ds_am[f'h_doy'], dim='year'), dim='year')
_lst = []
for v in drivers:
    driver_doy_p = rayleightest(doy_to_circ(ds_am[f'{v}_doy'], dim='year'), dim='year')
    doy_diff_sign = np.logical_and(wse_doy_p<=0.05, driver_doy_p<=0.05).sum('ensemble') == ds_am['ensemble'].size
    doy_diff_sign.name = f'{v}_doy_diff_sign'
    doy_diff_ensemble = mean_flood_day_diff(ds_am[f'{v}_doy'], ds_am['h_doy'])
    doy_diff = circ_mean(doy_to_circ(doy_diff_ensemble, dim=None), dim='ensemble') * 365 / (2*np.pi)
    doy_diff.name = f'{v}_doy_diff'
    doy_corr = corr_flood_day(ds_am[f'{v}_doy'], ds_am['h_doy']).mean('ensemble')
    doy_corr.name = f'{v}_doy_corr'
    _lst.extend([doy_diff, doy_corr, doy_diff_sign])
    
ds_out = xr.merge(_lst)
gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize', 'Q_mean']],
    ds_out.reset_coords(drop=True).to_dataframe()
], axis=1))

In [ ]:
wdw = 10
in_wdw = (gdf[[f'{v}_doy_diff' for v in drivers]].abs()<=wdw).sum(axis=1)
gdf['doy2'] = in_wdw>=2
gdf['doy1'] = in_wdw==1
gdf['doy0'] = in_wdw==0

In [ ]:
label=f'Difference mean annual flood day [days]'
var = 'doy_diff'
vmin, vmax, n= -14, 14, 8
cticks=[-183] + np.linspace(vmin, vmax, n).tolist() + [183]
cmap = plt.cm.RdYlBu

# label=f'Pearson correlation annual flood day [-]'
# var = 'doy_corr'
# vmin, vmax, n= 0.5, 1, 3
# cticks=np.linspace(vmin, vmax, n)
# cmap = plt.cm.Blues

plt.close('all')
fig = plt.figure(figsize=(15, 12))
grid = plt.GridSpec(3, 1, wspace=0.04, hspace=0.01)

for i, driver in enumerate(drivers):
    column = f'{driver}_{var}'
#     gdf1 = gdf.sort_values(by=column, ascending=True)
    gdf0 = gdf[gdf[f'{column}_sign']==0]
    gdf1 = gdf[gdf[f'{column}_sign']>0]
    gdf1 = gdf1[gdf1[f'doy2']>0]
    gdf1['abs_diff'] = np.abs(gdf1[column])
    gdf1 = gdf1.sort_values(by='abs_diff', ascending=False)
    axg = fig.add_subplot(grid[i,0], projection=crs_sub)
    basemap(axg, bbox=(-180, -60, 180, 90), gridlines=False, outline=False, **bmap_kwargs)
    gdf0.plot(ax=axg, marker='x', color='grey', markersize=10, linewidth=0.5, legend=False)
    plot_kwargs.update(markersize=12, marker='o', label=None)
    cax = plot_choropleth(
        fig, axg, gdf1, column=column, 
        cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=True,
        plot_kwargs=plot_kwargs,
        cbar_kwargs=dict(label=label, location='bottom', extend='none') if i == 2 else None ,
        cbar_pos = dict(pad=0.02, fraction=0.04, shrink=0.6) if i == 2 else None
    )
    axg.text(-175, 85, f'{letters[i]}) {driver_names[driver]}', transform=crs_sub, fontsize='large')

# fn_fig = join(fdir, f'single_drivers.png')
# print(basename(fn_fig))
# plt.savefig(fn_fig)

## drivers spearman rank correlation

In [ ]:
import scipy
def spearmanr(da0, da1, dim='time'):
    def _spearmanr(a, b):
        return np.asarray(scipy.stats.spearmanr(a,b))
    # apply_ufunc parameters
    kwargs = dict(               
        input_core_dims=[[dim], [dim]], 
        output_core_dims=[['stats']],
        dask='parallelized',
        output_dtypes=[float],    
        output_sizes={'stats': 2}, # on output, <dim> is reduced to length q.size 
        vectorize=True
    )
    da_out = xr.apply_ufunc(_spearmanr, da0, da1, **kwargs)
    da_out['stats'] = xr.Variable('stats', ['r', 'p'])
    return da_out.sel(stats='r').drop('stats'), da_out.sel(stats='p').drop('stats')

In [ ]:
wdw = 2
drivers = ['Hsurge','Q']

fn_peaks_wdw = join(ddir, f'rivmth_h_am_wdw{wdw}.nc')
ds_peaks = xr.open_dataset(fn_peaks_wdw)#.sel(ensemble=['anu'])

ds_spear = ds_peaks.coords.to_dataset().drop(['year'])
for d in drivers:
    ds_spear[f'{d}_r'], ds_spear[f'{d}_p'] = spearmanr(ds_peaks['h'], ds_peaks[d], dim='year')

N = ds_spear['ensemble'].size
Hsign = np.logical_and(ds_spear[f'{drivers[0]}_r']>=0, ds_spear[f'{drivers[0]}_p']<=0.05).sum('ensemble') >= N
Qsign = np.logical_and(ds_spear[f'{drivers[1]}_r']>=0, ds_spear[f'{drivers[1]}_p']<=0.05).sum('ensemble') >= N
Hsign1 = np.logical_and(ds_spear[f'{drivers[0]}_r']>=0, ds_spear[f'{drivers[0]}_p']<=0.05).sum('ensemble') >= 3
Qsign1 = np.logical_and(ds_spear[f'{drivers[1]}_r']>=0, ds_spear[f'{drivers[1]}_p']<=0.05).sum('ensemble') >= 3

ds_stats = ds_peaks.coords.to_dataset().drop(['year', 'ensemble'])
ds_stats['H_sign'] = Hsign
ds_stats['Q_sign'] = Qsign
ds_stats['H_r'] = ds_spear[f'{drivers[0]}_r'].mean('ensemble')
ds_stats['Q_r'] = ds_spear[f'{drivers[1]}_r'].mean('ensemble')
ds_stats['H and Q (N=5)'] = np.logical_and(Hsign, Qsign)
ds_stats['H and Q (N>=3)'] = np.logical_and(Hsign1, Qsign1)
ds_stats['H'] = np.logical_and(Hsign, ~Qsign)
ds_stats['Q'] = np.logical_and(~Hsign, Qsign)
ds_stats['insignificant'] = np.logical_and(~Hsign, ~Qsign)
ds_stats[['H and Q (N=5)', 'H and Q (N>=3)', 'H', 'Q', 'insignificant']].sum()

In [ ]:
gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'Q_mean']],
    ds_stats.reset_coords(drop=True).to_dataframe()
], axis=1))
# gdf.head()

In [ ]:
from scipy.stats import rankdata
locs = [2884, 1930, 1618]
riv_names = {
    809: 'Mattapone', 
    1695: 'Weser',
    1930: 'Dal',
    1618: 'Volta',  
    2884: 'Ataran'
}

rps = [ 1, 2, 4, 8, 16, 32]
cmap = ListedColormap(google_turbo_data[50:])
cmap.set_over('black')
norm = BoundaryNorm(rps, cmap.N)

mmap = {
    'Hsurge': '^',
    'Q': 's',
}
cmmap = {
    'Hsurge': plt.cm.tab10.colors[0],
    'Q': plt.cm.tab10.colors[-1],
}
labs = {
    'Hsurge': '$H_{total}$',
    'Q': 'Q',
}
    



In [ ]:
plt.close('all')
model2='anu'
xlim = [0.9,40]
rps2 = [1, 2, 4, 8, 16, 32]
shape=(2,len(locs))
fig = plt.figure(figsize=(len(locs)*4, 8.5))
grid = plt.GridSpec(*shape, hspace=0.15, wspace=0.15)

kwargs = dict(cmap=cmap, norm=norm)

for i, loc in enumerate(locs):
    name = riv_names[loc]
    gdf_loc = gdf.loc[loc,:]    
    irow = i // shape[1]
    icol = i % shape[1]
    
    # get data
    df_loc = ds_peaks.sel(index=loc, ensemble=model2).reset_coords(drop=True).to_dataframe()
    df_loc[~np.isfinite(df_loc)] = 0.9

    # plot rp
    ax = fig.add_subplot(grid[irow, icol])
    df_loc['h_rp2'] = 36/rankdata(-df_loc['h'])
    im = ax.scatter(df_loc[f'{drivers[0]}_rp'], df_loc[f'{drivers[1]}_rp'], c=df_loc['h_rp'], **kwargs)
    
    # plot rank correlation
    ax1 = fig.add_subplot(grid[irow+1, icol])
    for v in ['Hsurge', 'Q']:
        ax1.plot(rankdata(df_loc[v]), rankdata(df_loc['h']), color=cmmap[v], label=labs[v], marker=mmap[v], linewidth=0)
    ax1.plot([0, 41],[0, 41],'--k')
    
    # axis
    ax.set_xlim([0.85, 38])
    ax.set_ylim([0.85, 38])
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_aspect('equal')
    ax.grid(False)
    ax1.set_xlim([0.5, 41])
    ax1.set_ylim([0.5, 41])
    ax1.set_aspect('equal')
    ax1.grid(False)
    
    # labels
    if i == 0:
        ax.set_yticks(rps)
        ax.set_yticklabels(rps)
        ax.set_ylabel('$H_{total}$ return period [years]')
        ax1.set_ylabel('riverine water level rank [-]')
    else:
        ax.set_yticklabels([])
        ax.set_ylabel("")
        ax1.set_yticklabels([])
        ax1.set_ylabel("")
    ax.set_xticks(rps)
    ax.set_xticklabels(rps)
    ax.set_xlabel('Q return period [years]')
    ax1.set_xlabel('driver rank [-]')
    ax.set_title(f'{letters[i]}. {name} River')
    
    # text 
    Hr = float(ds_spear.sel(index=loc, ensemble=model2)[f'{drivers[0]}_r'].values)
    Qr = float(ds_spear.sel(index=loc, ensemble=model2)[f'{drivers[1]}_r'].values)
    Hp = float(ds_spear.sel(index=loc, ensemble=model2)[f'{drivers[0]}_p'].values)
    Qp = float(ds_spear.sel(index=loc, ensemble=model2)[f'{drivers[1]}_p'].values)
    ax1.text(1, 36, '$H_{total}$'+f': {Hr:.2f} ({Hp:.2f})\nQ: {Qr:.2f} ({Qp:.2f})')
       
# make colorbar
pad, shrink, fraction = 0.02, 1.0, 0.04
cax = fig.add_axes([1, 1, 0.1, 0.1]) # new ax
cbar = fig.colorbar(im, extend='max', cax=cax)
cbar.ax.set_ylabel("riverine water level\n return period [years]", rotation='vertical')
posn = ax.get_position()
cax.set_position([posn.x1+pad, posn.y0+posn.height*(1-shrink)/2., posn.width*fraction, posn.height*shrink])

ax1.legend(loc='lower left', bbox_to_anchor=(1.05, 0.5), fontsize='large', title='driver')

posn = ax1.get_position()
axg = fig.add_axes([posn.x1+0.01, posn.y0-0.1, posn.width*0.8, posn.height], projection=crs_sub) # new ax   
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=False, outline=False, features=['land'])
gdf.loc[locs,:].plot(ax=axg, marker='o', color='red', markersize=50, legend=True)
for i, loc in enumerate(locs):
    x, y = gdf.loc[loc,:].geometry.coords[0]
    axg.text(x+10, y, letters[i], transform=crs_sub)


fn_fig = join(fdir, f'locs_rp_spearmanr_{model2}_wdw{wdw}.png')
print(basename(fn_fig))
plt.savefig(fn_fig)

In [ ]:
label=f'spearman correlation [-]'
s=20
vmin, vmax, n= 0.5, 1, 6
cticks=np.linspace(vmin, vmax, n)
cmap  = plt.cm.Blues

clist = plt.cm.tab10.colors
mmapp = {'H and Q (N=5)': 'd', 'H and Q (N>=3)': 'd', 'H': '^', 'Q': 'o', 'insignificant': 'x'}
cmapp = {'H and Q (N=5)': clist[3], 'H and Q (N>=3)': clist[1], 'H': clist[0], 'Q': clist[-1], 'insignificant': 'grey'}
nmapp = {'H': 'Total barotropic sea leval variation ($H_{total}$)', 'Q': 'Discharge (Q)'}


plt.close('all')
fig = plt.figure(figsize=(18, 10.5))
grid = plt.GridSpec(3, 2, wspace=0.02, hspace=0.01)
        
# main map
axg = fig.add_subplot(grid[:-1, :], projection=crs_sub)
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=False, outline=False, **bmap_kwargs)
for d in list(mmapp.keys())[::-1]:
    gdf[gdf[d]==True].plot(ax=axg, marker=mmapp[d], color=cmapp[d], markersize=s, linewidth=0.5, legend=False, label=d)
axg.legend(title='pos. Spearman corr.', loc='lower left', bbox_to_anchor=(0.05, 0.1))
axg.text(-175, 85, f'{letters[0]}. main driver', transform=crs_sub, fontsize='large')

for i, v in enumerate(['H', 'Q']):
    # main map
    axg = fig.add_subplot(grid[-1, i], projection=crs_sub)
    basemap(axg, bbox=(-180, -60, 180, 90), gridlines=False, outline=False, **bmap_kwargs)
    column = f'{v}_r'
    gdf0 = gdf[gdf[f'{v}_sign']==False]
    gdf1 = gdf[gdf[f'{v}_sign']==True].sort_values(by=column, ascending=True)
    gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.6), markersize=12, linewidth=0.2, alpha=0.6, legend=False)
    plot_kwargs.update(markersize=12)
    cax = plot_choropleth(
        fig, axg, gdf1, column=column, 
        cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
        plot_kwargs=plot_kwargs,
        cbar_kwargs=dict(label=label, location='right', extend='min') if i == 1 else None,
        cbar_pos = dict(pad=0.02, fraction=0.015, shrink=0.8) 
    )
    axg.text(-175, 85, f'{letters[i+1]}. {nmapp[v]}', transform=crs_sub, fontsize='large')
fn_fig = join(fdir, f'spearman_driver_Hsurge_wdw{wdw}.png')
print(basename(fn_fig))
plt.savefig(fn_fig)

In [ ]:
dim='ensemble'


N= ds_stats[dim].size
rsign = ((ds_stats.sel(stats='p').drop('stats')<=0.05).sum(dim) >= N).rename({f'{v}_spearman': f'{v}' for v in drivers})
rmean = ds_stats.sel(stats='r').drop('stats').mean(dim).rename({f'{v}_spearman': v for v in drivers})
ds_stats = xr.concat([rmean, rsign], dim='stat')
ds_stats = xr.concat([ds_stats[d] for d in drivers], dim='driver')
ds_stats.name = 'spearmanr'
ds_stats['stat'] = xr.Variable('stat', ['mean', 'sign'])
ds_stats['driver'] = xr.Variable('driver', drivers)


da_drivers = xr.DataArray(dims=('driver'), data=drivers)
main_driver = da_drivers.isel(driver = ds_stats.sel(stat='mean').drop('stat').argmax('driver'))
main_driver.name = 'main_driver'
ds_stats.sel(driver=main_driver)

rmean_main = ds_stats.sel(driver=main_driver).sel(stat='mean').drop('stat')
rmean_main.name = 'main_r'
rsign_main = ds_stats.sel(driver=main_driver).sel(stat='sign').drop('stat')
rsign_main.name = 'main_sign'

ds_out = xr.merge([rmean, rsign.rename({f'{v}': f'{v}_sign' for v in drivers}),  rmean_main, rsign_main, main_driver, ])

gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize', 'Q_mean']],
    ds_out.reset_coords(drop=True).to_dataframe()
], axis=1))
gdf0 = gdf[gdf['main_sign']==0]
gdf1 = gdf[gdf['main_sign']==1]
gdf_s = gdf1[gdf1['main_driver']=='Hskewsurge']
gdf_q = gdf1[gdf1['main_driver']=='Qmsl']
gdf_t = gdf1[gdf1['main_driver']=='Htot']
gdf1.index.size

In [ ]:
# wdw = 1
# drivers = ['Htide', 'Hskewsurge', 'Q']
# model='mean'
# column = 'h_ratio'
# fn_peaks_wdw = join(ddir, f'rivmth_AMpeaks_wdw{wdw}.nc')
# fn_peaks_wdw_stats = join(ddir, f'rivmth_AMpeaks_wdw{wdw}_stats.nc')
# ds = xr.open_dataset(fn_peaks_wdw)
# ds_stats = xr.open_dataset(fn_peaks_wdw_stats).rename({'sign_a05': 'sign'})
# ds['h'] = ds['WSE']-ds['z0']
# ds_stats['h_ratio'] = (ds_stats['h_driver'] / ds_stats['h_actual']) * 100

# ds_ratio  = (ds['h'].sel(driver=drivers).mean('rank') / ds['h'].sel(driver='WSE').mean('rank')) * 100
# ds_ratio = xr.merge([
#     ds_ratio.sel(driver=d).drop('driver').to_dataset().rename({'h': f'h_ratio_{d}'}) for d in drivers
# ])

# if model != 'mean':
#     ds_stats = ds_stats.sel(ensemble=[model])
#     ds_ratio = ds_ratio.sel(ensemble=[model])
#     ds = ds.sel(ensemble=[model])


# N = ds_stats['ensemble'].size
# da_drivers = xr.DataArray(dims=('driver'), data=drivers)
# main_driver = da_drivers.isel(driver = xr.concat([
#     (ds_stats['driver']==driver).sum('ensemble')
#     for driver in drivers
# ], dim='driver').argmax('driver'))
# main_driver.name = 'driver'
# h_reldiff = ds_stats[column].mean('ensemble')
# sign = ds_stats['sign'].sum('ensemble') >= N
# # sign = (ds_stats['h_reldiff'] >= 1).sum('ensemble') >= N
# # sign.name = 'sign'
# ds_ratio = ds_ratio.mean('ensemble').reset_coords(drop=True)
# ds_out = xr.merge([main_driver, sign, h_reldiff, ds_ratio])

# gdf = pandas2geopandas(pd.concat([
#     attrs[['lat', 'lon', 'markersize', 'Q_mean']],
#     ds_out.to_dataframe()
# ], axis=1)).sort_values(by=column, ascending=True)
# gdf0 = gdf[gdf['sign']==False]
# gdf1 = gdf[gdf['sign']]
# gdf_s = gdf1[gdf1['driver']=='Hskewsurge']
# gdf_q = gdf1[gdf1['driver']=='Q']
# gdf_t = gdf1[gdf1['driver']=='Htide']

# df_box = ds_stats[column].sel(index=gdf1.index).to_series().unstack(-1).T
# print(gdf1.index.size)
# gdf.head()

In [ ]:
label=f'spearman correlation [-]'
s=25
vmin, vmax, n= 0.5, 1, 6
cticks=np.linspace(vmin, vmax, n)
cmap  = plt.cm.Blues_r

plt.close('all')
fig = plt.figure(figsize=(14, 7))
# grid = plt.GridSpec(4, 3, wspace=0.04, hspace=0.01)

# main map
column = 'main_r'
axg = fig.add_subplot(projection=crs_sub)
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=False, outline=False, **bmap_kwargs)
gdf0.plot(ax=axg, marker='x', color='grey', markersize=15, linewidth=0.5, legend=False)
plot_kwargs.update(markersize=s, marker='o', label='discharge')
cax = plot_choropleth(
    fig, axg, gdf_q, column=column, 
    cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
    plot_kwargs=plot_kwargs,
    cbar_kwargs=dict(label=label, location='right'), #, extend='max'),
    cbar_pos = dict(pad=0.02, fraction=0.01, shrink=0.6)
)
plot_kwargs.update(markersize=s, marker='^', label='surge')
plot_choropleth(
    fig, axg, gdf_s, column=column, 
    cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
    plot_kwargs=plot_kwargs, cbar_kwargs=False,
)
plot_kwargs.update(markersize=s, marker='d', label='tide')
plot_choropleth(
    fig, axg, gdf_t, column=column, 
    cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
    plot_kwargs=plot_kwargs, cbar_kwargs=False,
)
axg.legend(title='main driver', loc='lower left', bbox_to_anchor=(0.05, 0.1))

# fn_fig = join(fdir, f'main_driver.png')
# print(basename(fn_fig))
# plt.savefig(fn_fig)

In [ ]:
label=f'spearman correlation [-]'

vmin, vmax, n= 0.25, 1, 4
cticks=np.linspace(vmin, vmax, n)
cmap = plt.cm.Blues
plt.close('all')
fig = plt.figure(figsize=(12, 10))
grid = plt.GridSpec(3, 1, wspace=0.04, hspace=0.01)

for i, driver in enumerate(drivers):
    column = f'{driver}'
#     gdf0 = gdf[gdf[f'{column}_sign']==0]
    gdf1 = gdf[gdf[f'{column}_sign']>=0].sort_values(by=column, ascending=True)

    axg = fig.add_subplot(grid[i,0], projection=crs_sub)
    basemap(axg, bbox=(-180, -60, 180, 90), gridlines=False, outline=False, **bmap_kwargs)
    gdf0.plot(ax=axg, marker='x', color='grey', markersize=10, linewidth=0.5, legend=False)
    plot_kwargs.update(markersize=15, marker='o')
    plot_kwargs.pop(label, None)
    cax = plot_choropleth(
        fig, axg, gdf1, column=column, 
        cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
        plot_kwargs=plot_kwargs,
        cbar_kwargs=dict(label=label, location='bottom', extend='min') if i == 2 else None ,
        cbar_pos = dict(pad=0.02, fraction=0.04, shrink=0.6) if i == 2 else None
    )
    axg.text(-175, 85, f'{letters[i]}) {driver_names[driver]}', transform=crs_sub, fontsize='large')

# fn_fig = join(fdir, f'single_drivers.png')
# print(basename(fn_fig))
# plt.savefig(fn_fig)

In [ ]:
label=f'percentage of mean annual max. water level \nexplained by a single driver [%]'

vmin, vmax, n= 85, 100, 5
# vmin, vmax, n= -0.1, 0.25, 8
cticks=np.linspace(vmin, vmax, n)
cmap_turbo2 = ListedColormap([
    interpolate_cmap(google_turbo_data, x) for x in np.linspace(1, 0.25, 100)
])
#     np.hstack([np.linspace(0.4-(.4*abs(vmin)/vmax)*2, .4, int(-vmin*100)), np.linspace(0.55, 1, int(vmax*100))])
# ])
# cmap = plt.cm.Greens
cmap = cmap_turbo2
# cmap.set_over('grey')
plt.close('all')
fig = plt.figure(figsize=(19, 11))
grid = plt.GridSpec(3, 6, hspace=0.0, wspace=0.25)

# main map
column = 'h_ratio'
axg = fig.add_subplot(grid[:-1, :-1], projection=crs_sub)
add_regions(axg, regions, dx=-6, dy=-6)
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
gdf0.plot(ax=axg, marker='x', color='grey', markersize=5, linewidth=0.5, legend=False)
plot_kwargs.update(markersize=s, marker='o')
cax = plot_choropleth(
    fig, axg, gdf_q, column=column, 
    cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
    plot_kwargs=plot_kwargs,
    cbar_kwargs=dict(label=label, location='right'), #, extend='max'),
    cbar_pos = dict(pad=0.08, fraction=0.1)
)
plot_kwargs.update(markersize=s, marker='^')
plot_choropleth(
    fig, axg, gdf_s, column=column, 
    cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
    plot_kwargs=plot_kwargs, cbar_kwargs=False,
)
plot_kwargs.update(markersize=s, marker='d')
plot_choropleth(
    fig, axg, gdf_t, column=column, 
    cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
    plot_kwargs=plot_kwargs, cbar_kwargs=False,
)

# submaps 
axs = dict()
# gdf1 = gdf1.sort_values('markersize', ascending=True)
for i, (name, bbox) in enumerate(regions.items()):
    
    ax = fig.add_subplot(grid[-1, i*2:(i+1)*2], projection=crs_sub)
    basemap(ax, bbox=bbox, gridlines=False, gridspacing=10, outline=True, **bmap_kwargs)
    gdf0.plot(ax=ax, marker='x', color='grey', markersize=20, linewidth=0.5, legend=False)
    plot_kwargs.update(markersize=s*2, marker='o')
    plot_choropleth(
        fig, ax, gdf_q, column=column, 
        cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
        plot_kwargs=plot_kwargs, cbar_kwargs=False
    )
    plot_kwargs.update(markersize=s*2, marker='^')
    plot_choropleth(
        fig, ax, gdf_s, column=column, 
        cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
        plot_kwargs=plot_kwargs, cbar_kwargs=False,
    )
    plot_kwargs.update(markersize=s*2, marker='d')
    plot_choropleth(
        fig, ax, gdf_t, column=column, 
        cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
        plot_kwargs=plot_kwargs, cbar_kwargs=False,
    )
    xmin, ymin, xmax, ymax = bbox
    ax.text(xmin+(xmax-xmin)*0.03, ymin+(ymax-ymin)*0.91, name, transform=crs_sub, fontsize='large')
    axs[name] = ax

# boxplot ensemble

ax = fig.add_subplot(grid[:-1, -1:])
box_kwargs.update(widths=0.5)
color_props = dict(boxes="lightgrey", whiskers="k", medians="k", caps="k")
df_box.plot.box(ax=ax, color=color_props, patch_artist=True, **box_kwargs)
ax.set_ylim([vmin, vmax])
ax.yaxis.set_ticks(cticks)
ax.yaxis.set_ticklabels([])
ax.grid(axis='x')
ax.set_xlabel('model')
cpos = cax.get_position()
bpos = ax.get_position()
cax.set_position([cpos.x0, cpos.y0, cpos.width, cpos.height])
# ax.set_position([cpos.x0+0.03, cpos.y0, bpos.width*1.5, cpos.height])
ax.set_position([cpos.x0+0.03, cpos.y0, bpos.width*1.5, cpos.height/1.05])
cax.yaxis.set_ticks_position('left')
cax.yaxis.set_label_position('left')
        
# fn_fig = join(fdir, f'reldiff_mam_wdw{wdw}_{model}.png')
# print(basename(fn_fig))
# plt.savefig(fn_fig)

In [ ]:
plt.close('all')

mmap = {
    'Hskewsurge': '^',
    'Htide': 'd',
    'Q': 's',
    'WSE': 'o',
}
cmmap = {
    'Hskewsurge': 'blue',
    'Htide': 'cyan',
    'Q': 'green',
    'WSE': 'k',
}
labs = {
    'WSE': 'local water level',
    'Hskewsurge': 'skew surge',
    'Htide': 'tide',
    'Q': 'discharge',
}

xlim = [1979,2015]
yrs = np.arange(1980,2015,10)
drivers_rp = ['WSE_rp'] + [f'{d}_rp' for d in drivers]
shape=(2,3)
fig = plt.figure(figsize=(15, 8))
grid = plt.GridSpec(*shape, hspace=0.1, wspace=0.2)
axes =[]
for irow in range(shape[0]):
    for icol in range(shape[1]):
        axes.append(fig.add_subplot(grid[irow, icol]))
    
posn = axes[2].get_position()
axg = fig.add_axes([posn.x1+0.01, posn.y0-0.25, posn.width*0.8, posn.height], projection=crs_sub) # new ax   
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=False, outline=False, features=['land'])
gdf.loc[locs,:].plot(ax=axg, marker='o', color='red', markersize=50, legend=True)

for i, loc in enumerate(locs):
    name = riv_names[loc]
    gdf_loc = gdf.loc[loc,:]
    driver = gdf_loc['driver']
    ds_loc = ds[drivers_rp + ['h', 'WSE', 'time']].sel(index=loc, ensemble=model)
    df_ = ds_loc.reset_coords(drop=True).to_dataframe()
    df_['year'] = df_['time'].dt.year
    
    irow = i // shape[1]
    icol = i % shape[1]
    ax = axes[i]
    for d in ['WSE'] + drivers:
        df_.loc[(d,),:].plot.scatter(
            ax=ax, y='h', x='year', 
            marker=mmap[d], 
            color=cmmap[d], 
            s=50 if d == 'WSE' else 20,
            label=labs[d] if i == 5 else None, 
            zorder=1
        )
        havg = df_.loc[(d,),'h'].mean()
        ax.plot(xlim, [havg, havg], color=cmmap[d], linestyle='--', zorder=0, alpha=0.8)
    
    ax.set_xticks(yrs)
    if irow == 1:
        ax.set_xticklabels(yrs)
        ax.set_xlabel(f'year')
    else:
        ax.set_xticklabels([])
        ax.set_xlabel('')

    if icol == 0:
        t = labs['WSE']
        ax.set_ylabel(f'{t} [m]')
    else:
        ax.set_ylabel('')

    l = letters[i]
    r = gdf_loc[column]
    x, y = gdf_loc.geometry.coords[0]
    axg.text(x+10, y, letters[i], transform=crs_sub)
    ax.set_xlim(xlim)
    ymin, ymax = ax.get_ylim()
    dname = labs[driver].split('-')[0]
    ax.text(xlim[0], ymax+(ymax-ymin)*0.01, f'{l}. {name} - min. $\Delta$ h: {r:.1f}%', fontsize='large')
    ax.grid(False)

ax.legend(loc='lower left', bbox_to_anchor=(1.05, 0.05), fontsize='large', title='events')

fn_fig = join(fdir, f'locs_reldiff_mam_wdw{wdw}_{model}.png')
print(basename(fn_fig))
plt.savefig(fn_fig)

In [ ]:
from skill_stats import spearman_rank_correlation
ds_loc = ds[drivers_rp + ['time']].sel(driver='WSE', index=locs[2])
spearman_rank_correlation(ds_loc['WSE_rp'], ds_loc['Q_rp'], dim='rank')

In [ ]:
plt.close('all')

xlim = [0.2,40]
rps2 = [0.1, 0.2, 0.4, 1, 2, 4, 8, 16, 32]
drivers_rp = ['WSE_rp'] + [f'{d}_rp' for d in drivers]

shape=(2,3)
fig = plt.figure(figsize=(15, 8))
grid = plt.GridSpec(*shape, hspace=0.1, wspace=0.2)
axes =[]
for irow in range(shape[0]):
    for icol in range(shape[1]):
        axes.append(fig.add_subplot(grid[irow, icol]))
    
posn = axes[2].get_position()
axg = fig.add_axes([posn.x1+0.01, posn.y0-0.25, posn.width*0.8, posn.height], projection=crs_sub) # new ax   
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=False, outline=False, features=['land'])
gdf.loc[locs,:].plot(ax=axg, marker='o', color='red', markersize=50, legend=True)

for i, loc in enumerate(locs):
    name = riv_names[loc]
    gdf_loc = gdf.loc[loc,:]
    driver = gdf_loc['driver']
    ds_loc = ds[drivers_rp + ['h', 'WSE', 'time']].sel(index=loc, driver='WSE', ensemble=model)
    df_ = ds_loc.reset_coords(drop=True).to_dataframe()
    df_['year'] = df_.loc[:,'time'].dt.year
    df_['rp'] = 36. / df_.index
    
    irow = i // shape[1]
    icol = i % shape[1]
    ax = axes[i]
    for d in drivers:
        df_.plot.scatter(
            ax=ax, x='WSE_rp', y=f'{d}_rp', 
            marker=mmap[d], 
            color=cmmap[d] if d == 'WSE' else 'black', 
            s=25,
            label=labs[d] if i == 5 else None,
            zorder=1
        )
    ax.plot(xlim, xlim, 'k--', zorder=0) #, label='1:1')
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_yticks(rps2)
    ax.set_xticks(rps2)
    if icol == 0:
        ax.set_yticklabels(rps2)
        ax.set_ylabel(f'driver rp [year]')
    else:
        ax.set_yticklabels([])
        ax.set_ylabel('')

    if irow == 1:
        ax.set_xlabel('wl event rp [year]')
        ax.set_xticklabels(rps2)
    else:
        ax.set_xlabel('')
        ax.set_xticklabels([])
    
    ax.set_ylim(xlim)
    ax.set_xlim(xlim)
    
    l = letters[i]
    r = gdf_loc[column]
    ymin, ymax = ax.get_ylim()
    ax.text(xlim[0], ymax+(ymax-ymin)*0.1, f'{l}. {name} River', fontsize='large')
    x, y = gdf_loc.geometry.coords[0]
    axg.text(x+10, y, letters[i], transform=crs_sub)
    
    ax.grid(False)

ax.legend(loc='lower left', bbox_to_anchor=(1.05, 0.05), fontsize='large', title='drivers')

fn_fig = join(fdir, f'locs_rp_rp_wdw{wdw}_{model}.png')
print(basename(fn_fig))
plt.savefig(fn_fig)

In [ ]:
attrs2 = pd.concat([
    attrs,
    ds_out.to_dataframe()
], axis=1)
attrs2['Q_amax_log10'] = np.log10(np.maximum(attrs2['Q_amax'].values, 0.001))
attrs2['uparea_log10'] = np.log10(np.maximum(attrs2['uparea'].values, 0.001))
attrs2.columns

In [ ]:
from scipy.stats import gaussian_kde
qmin=0
vs = {
#     'Htiderange_amax': dict(lim=[0, 9.9], label='mean an. max\n tidal range [m]'), 
    'Hskewsurge_amax': dict(lim=[0, 1.05], label='mean an. max\nskew surge [m]', xticks=np.arange(0,1.01,.2), fmt='{:.1f}'),
#     'Hseas_amax': dict(lim=[0, 0.25], label='mean an. max\nHseas [m]', xticks=np.arange(0,0.26,0.05), fmt='{:.1f}'),
    'Q_amax_log10': dict(lim=[np.log10(np.max([qmin,10])), 5], label='mean an. max\ndischarge [m$^3$/s]', xticks=range(1,5), fmt='10$^{:d}$'),
    'uparea_log10': dict(lim=[3, 5.5], label='catchment area [km$^2$]', xticks=range(3,6), fmt='10$^{:d}$'),
}
hist_kws={"histtype": "step", "linewidth": 1.5, "alpha": 1, "color": "m"}
cmax= {0: 300, 10: 200}
var = 'h_reldiff'

attrs_sign = attrs2 #[attrs2['sign']==1]
vmin, vmax = np.percentile(attrs_sign[var].values, [1, 99])
vmin, vmax = np.maximum(vmin, -0.1), np.maximum(vmax, 0.2)
palette= "ch:start=.0, rot=-.75"
cmap_dens = ListedColormap(sns.color_palette(palette, 100).as_hex())
fig, axs = plt.subplots(1, len(vs), figsize=(len(vs)*4+1, 4), sharey=True, gridspec_kw=dict(hspace=0.1))

# density
# norm=plt.Normalize(vmin=0, vmax=1)
# sm = plt.cm.ScalarMappable(cmap=cmap_dens, norm=norm)

y = attrs_sign[var].values
for i, (ax, v) in enumerate(zip(axs, vs)):
    # Calculate the point density
    x = attrs_sign[v].values.copy()
    xy = np.vstack([x,y.copy()])
    z = gaussian_kde(xy)(xy)
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]
    sns.scatterplot(x=x, y=y, 
                    hue=z, palette=palette,
#                             color='b',
                    ax=ax, linewidth=0.0, 
                    s=25, 
                    legend=False)
    ax.set_xlim(vs[v]['lim'])
    if 'xticks' in vs[v]:
        ax.set_xticks(vs[v]['xticks'])
        ax.set_xticklabels([vs[v]['fmt'].format(x) for x in vs[v]['xticks']])
    ax.set_xlabel(vs[v]['label'])
    ax.text(0.05, 0.95, letters[i], fontsize='large',
           horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, )
    if i == 0:
        ax.set_ylabel(label)
        ax.set_ylim([vmin, vmax])
    if i == (len(axs)-1):
        norm=plt.Normalize(vmin=0, vmax=1)
        sm = plt.cm.ScalarMappable(cmap=cmap_dens, norm=norm)
        the_divider = make_axes_locatable(ax)
        caxis = the_divider.append_axes("right", size="5%", pad=0.1)
        sm._A = []
        cbar = plt.colorbar(sm, cax=caxis)
        cbar.set_label('relative density [-]')
        cbar.set_ticks([0,1])
        cbar.ax.set_yticklabels(['low', 'high'], rotation=90, fontdict=dict(va='center'))

fn = join(fdir, f'attrs_{var}_{model}_all.png')
plt.savefig(fn)
print(basename(fn))

## difference rp scenarios

In [ ]:
alpha=0.025
model='mean'
fn_peaks_rp_ci = join(ddir, f'rivmth_peaks_gumb_ci_N1e4.nc')
fn_peaks_rp = join(ddir, f'rivmth_peaks_gumb.nc')
da_wse_rps = xr.open_dataset(fn_peaks_rp)['WSE_ev']
da_wse_rps_ci = xr.open_dataset(fn_peaks_rp_ci)['WSE_ev_ci']

if model!='mean':
    da_wse_rps = da_wse_rps.sel(ensemble=[model])
    da_wse_rps_ci = da_wse_rps_ci.sel(ensemble=[model])
    
# diff
diff_surge_seas = (da_wse_rps.sel(scen='surge') - da_wse_rps.sel(scen='seas'))
diff_surge_seas.name = 'diff_surge_seas'
diff_seas_tide = (da_wse_rps.sel(scen='seas') - da_wse_rps.sel(scen='tide'))
diff_seas_tide.name = 'diff_seas_tide'
diff_surge_tide = (da_wse_rps.sel(scen='surge') - da_wse_rps.sel(scen='tide'))
diff_surge_tide.name = 'diff_surge_tide'
ds_diff = xr.merge([
    diff_surge_seas,
    diff_seas_tide,
    diff_surge_tide
])

dim = 'ensemble'
# average and calculate significance based on std error
N = ds_diff[dim].size
ds_diff_mean = ds_diff.mean(dim)
ds_diff_dir =  xr.ufuncs.fabs(xr.ufuncs.sign(ds_diff).sum(dim)) == N
# ds_diff_sign = xr.ufuncs.fabs(ds_diff_mean / ds_diff.std(dim)) > (2 / xr.ufuncs.sqrt(N-1))

ds_lst = []
for var in list(ds_diff.data_vars.keys()):
    v1,v2,v3 = var.split('_')
    da_sign = xr.where(
        xr.ufuncs.sign(ds_diff[var])>0,
        da_wse_rps_ci.sel(scen=v2, alpha=alpha) > da_wse_rps_ci.sel(scen=v3, alpha=1-alpha),
        da_wse_rps_ci.sel(scen=v3, alpha=alpha) > da_wse_rps_ci.sel(scen=v2, alpha=1-alpha),
    )#.drop('alpha')
    da_sign.name = var
    ds_lst.append(da_sign)
ds_diff_sign2 = xr.merge(ds_lst).sum('ensemble') >= N

ds_diff_stats = xr.concat([
    ds_diff_mean,
    np.logical_and(ds_diff_sign2, ds_diff_dir),
], dim='stat') 

ds_diff_stats['stat'] = xr.Variable('stat', ['mean', 'sign'])
(ds_diff_stats.sel(stat='sign', rp=2)==1).sum('index')

In [ ]:
var = 'diff_surge_tide'
for rp in [2, 10]:
    label=f'change in riverine water level \n{rp}-yr return period [m]'
    vmin, vmax, n= -0.2, 1.0, 7
    cticks=np.linspace(vmin, vmax, n)
    cmap_turbo_div = ListedColormap([
        interpolate_cmap(google_turbo_data, x) for x in 
        np.hstack([np.linspace(0.55-(.4*abs(vmin)/vmax)*2, .55, int(-vmin*20)), np.linspace(0.6, 1, int(vmax*20))])
    ])


    gdf = pandas2geopandas(pd.concat([
        attrs[['lat', 'lon', 'markersize']],
        ds_diff_stats[var].sel(rp=rp).to_series().unstack().T
    ], axis=1))

    plt.close('all')
    fig = plt.figure(figsize=(19, 11))
    grid = plt.GridSpec(3, 6, hspace=0.0, wspace=0.25)

    column = 'mean'
    idx_sign = gdf['sign']==1 #np.logical_and(gdf['sign']==1, np.abs(nanzscore(gdf['mean'].values))<=5)

    gdf0 = gdf[idx_sign==False].copy()
    gdf1 = gdf[idx_sign].copy().sort_values(column, ascending=True)
    df_box = ds_diff[var].sel(rp=rp, index=gdf1.index).to_series().unstack(-1).T

    # main map
    axg = fig.add_subplot(grid[:-1, :-1], projection=crs_sub)
    add_regions(axg, regions, dx=-6, dy=-6)
    basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
    gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.6), markersize=5, linewidth=0.2, alpha=0.6, legend=False)

    plot_kwargs.update(markersize=s, marker='o')
    cax = plot_choropleth(
        fig, axg, gdf1, column=column, 
        cmap=cmap_turbo_div, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
        plot_kwargs=plot_kwargs,
        cbar_kwargs=dict(label=label, location='right', extend='both'),
        cbar_pos = dict(pad=0.1, fraction=0.1)
    )

    # submaps 
    axs = dict()
    # gdf1 = gdf1.sort_values('markersize', ascending=True)
    for i, (name, bbox) in enumerate(regions.items()):
        ax = fig.add_subplot(grid[-1, i*2:(i+1)*2], projection=crs_sub)
        basemap(ax, bbox=bbox, gridlines=False, gridspacing=10, outline=True, **bmap_kwargs)
        gdf0.plot(ax=ax, marker='x', color=(0, 0, 0, 0.8), markersize=15, linewidth=0.2, legend=False)
        plot_kwargs.update(markersize=s*2, marker='o')
        plot_choropleth(
            fig, ax, gdf1, column=column, 
            cmap=cmap_turbo_div, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
            plot_kwargs=plot_kwargs, cbar_kwargs=False
        )
        xmin, ymin, xmax, ymax = bbox
        ax.text(xmin+(xmax-xmin)*0.03, ymin+(ymax-ymin)*0.91, name, transform=crs_sub, fontsize='large')
        axs[name] = ax

    # boxplot ensemble
    box_kwargs.update(widths=0.5)
    color_props = dict(boxes="lightgrey", whiskers="k", medians="k", caps="k")
    ax = fig.add_subplot(grid[:-1, -1:])
    df_box.plot.box(ax=ax, color=color_props, patch_artist=True, **box_kwargs)
#         ax.set_ylim([cticks[1], cticks[-2]])
#         ax.yaxis.set_ticks(cticks[1:-2])
    ax.set_ylim([vmin, vmax])
    ax.yaxis.set_ticks(cticks)
    ax.yaxis.set_ticklabels([])
    ax.grid(axis='x')
    ax.set_xlabel('model')
    cpos = cax.get_position()
    bpos = ax.get_position()
    cax.set_position([cpos.x0, cpos.y0, cpos.width, cpos.height])
    # ax.set_position([cpos.x0+0.03, cpos.y0, bpos.width*1.5, cpos.height])
    ax.set_position([cpos.x0+0.03, cpos.y0+cpos.height*0.05, bpos.width*1.5, cpos.height/1.10])
    cax.yaxis.set_ticks_position('left')
    cax.yaxis.set_label_position('left')

    fn_fig = join(fdir, f'rivmth_peaks_gumb_ci{alpha}_rp{rp}_{model}.png')
    print(basename(fn_fig))
    plt.savefig(fn_fig)

In [ ]:
vmin, vmax, n= -0.2, 1.0, 7
cticks=np.linspace(vmin, vmax, n)
cmap_turbo_div = ListedColormap([
    interpolate_cmap(google_turbo_data, x) for x in 
    np.hstack([np.linspace(0.55-(.4*abs(vmin)/vmax)*2, .55, int(-vmin*20)), np.linspace(0.6, 1, int(vmax*20))])
])
var = 'diff_surge_tide'
column='mean'

for rp in [2,10]:
    plt.close('all')
    fig = plt.figure(figsize=(17, 11))
    grid = plt.GridSpec(3, 2, hspace=0.0, wspace=0.1)
    label=f'change in riverine water level \n{rp}-yr return period [m]'

    gdf = pandas2geopandas(pd.concat([
        attrs[['lat', 'lon', 'markersize']],
        ds_diff_stats[var].sel(rp=rp).to_series().unstack().T
    ], axis=1))
    axg = fig.add_subplot(grid[:2,:], projection=crs_sub)
    gdf = pandas2geopandas(pd.concat([
        attrs[['lat', 'lon', 'markersize']],
        ds_diff_stats[var].sel(rp=rp).to_series().unstack().T
    ], axis=1))
    idx_sign = gdf['sign']==1 #np.logical_and(gdf['sign']==1, np.abs(nanzscore(gdf['mean'].values))<=5)
    gdf0 = gdf[idx_sign==False].copy()
    gdf1 = gdf[idx_sign].copy().sort_values('markersize', ascending=True)
    # main map
    basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
    gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.6), markersize=5, linewidth=0.2, alpha=0.6, legend=False)
    plot_kwargs.update(markersize=s, marker='o')
    cax = plot_choropleth(
        fig, axg, gdf1, column=column, 
        cmap=cmap_turbo_div, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
        plot_kwargs=plot_kwargs,
        cbar_kwargs=dict(label=label, location='right', extend='both'),
        cbar_pos = dict(pad=0.02, fraction=0.1)
    )
    axg.text(-175, 80, letters[0], transform=crs_sub, fontsize='large')

    # sub map
    for i, var in enumerate(['diff_surge_seas', 'diff_seas_tide']):
        axg = fig.add_subplot(grid[-1,i], projection=crs_sub)
        gdf = pandas2geopandas(pd.concat([
            attrs[['lat', 'lon', 'markersize']],
            ds_diff_stats[var].sel(rp=rp).to_series().unstack().T
        ], axis=1))
        idx_sign = gdf['sign']==1#np.logical_and(gdf['sign']==1, np.abs(nanzscore(gdf['mean'].values))<=5)
        gdf0 = gdf[idx_sign==False].copy()
        gdf1 = gdf[idx_sign].copy().sort_values('markersize', ascending=True)
        basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
        gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.6), markersize=5, linewidth=0.2, alpha=0.6, legend=False)
        plot_kwargs.update(markersize=s, marker='o')
        plot_choropleth(
            fig, axg, gdf1, column=column, 
            cmap=cmap_turbo_div, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
            plot_kwargs=plot_kwargs, cbar_kwargs=False
        )
        axg.text(-175, 75, letters[i+1], transform=crs_sub, fontsize='large')

    fig.tight_layout()
    fn_fig = join(fdir, f'scen_rivmth_peaks_gumb_ci{alpha}_rp{rp}_{model}.png')
    print(basename(fn_fig))
    plt.savefig(fn_fig)


In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../3-postprocess/'))
import xstats as xs 


In [ ]:
scen_cmap = {'surge': 'black', 'seas': 'blue', 'tide': 'cyan'}
scen_mmap  = {'surge': 'o', 'seas': '^', 'tide': 'd'}
# scen_labs  = {'surge': 'surge', 'tide': 'base'}
ds_ev = xr.open_dataset(fn_peaks_rp_ci)

In [ ]:
plt.close('all')

xlim = [0.9,40]
rps2 = [1, 2, 4, 8, 16, 32]
shape=(2,3)
fig = plt.figure(figsize=(15, 8))
grid = plt.GridSpec(*shape, hspace=0.1, wspace=0.2)
axes =[]
for irow in range(shape[0]):
    for icol in range(shape[1]):
        axes.append(fig.add_subplot(grid[irow, icol]))
    
posn = axes[2].get_position()
axg = fig.add_axes([posn.x1+0.01, posn.y0-0.25, posn.width*0.8, posn.height], projection=crs_sub) # new ax   
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=False, outline=False, features=['land'])
gdf.loc[locs,:].plot(ax=axg, marker='o', color='red', markersize=50, legend=True)

for i, loc in enumerate(locs):
    name = riv_names[loc]
    gdf_loc = gdf.loc[loc,:]    
    irow = i // shape[1]
    icol = i % shape[1]
    ax = axes[i]
    
    ds_ev_loc = ds_ev.sel(index=loc)
    T = ds_ev_loc.rp.values
    for scen in ['surge', 'seas', 'tide']:
        _ds = ds_ev_loc.sel(scen=scen, ensemble=model)
        Tpeaks_am, peaks_am = xs.weibull(_ds['WSE_am'].values)
        ev_ci = _ds['WSE_ev_ci'].sel(alpha=[alpha, 1-alpha]).values
        ev = _ds['WSE_ev'].values
        ax.scatter(Tpeaks_am, peaks_am, c=scen_cmap[scen], marker=scen_mmap[scen], label=scen, zorder=2)
        ax.plot(T, ev, color=scen_cmap[scen], zorder=1)
        ax.plot(T, ev_ci[0,:], color=scen_cmap[scen], linestyle='-.', alpha=0.4, zorder=0)
        ax.plot(T, ev_ci[1,:], color=scen_cmap[scen], linestyle='-.', alpha=0.4, zorder=0)
    ax.set_xscale('log')
    if irow == 1:
        ax.set_xticks(rps2)
        ax.set_xticklabels(rps2)
    else:
        ax.set_xticks(rps2)
        ax.set_xticklabels([])
    if icol == 0:
        ax.set_ylabel('WSE [m+EGM96]')
    ax.set_xlim(xlim)
    ax.grid(False)

    l = letters[i]
    r = gdf_loc[column]
    ymin, ymax = ax.get_ylim()
    ax.text(xlim[0], ymax+(ymax-ymin)*0.01, f'{l}. {name} River', fontsize='large')
    x, y = gdf_loc.geometry.coords[0]
    axg.text(x+10, y, letters[i], transform=crs_sub)
    
    ax.grid(False)

ax.legend(loc='lower left', bbox_to_anchor=(1.05, 0.05), fontsize='large', title='sea boundary')

fn_fig = join(fdir, f'locs_rivmth_peaks_gumb_ci{alpha}_{model}.png')
print(basename(fn_fig))
plt.savefig(fn_fig)

In [ ]:
fn_ev = join(ddir, 'rivmth_peaks_gumb_ci_N1e4.nc')
alpha = 0.025

ds_ev = xr.open_dataset(fn_ev)
da_wse_ev = ds_ev['WSE_ev']#.sel(ensemble='cnrs')
da_wse_ev_ci_high = ds_ev['WSE_ev_ci'].sel(alpha=1-alpha).squeeze() #.sel(ensemble='cnrs')
da_wse_ev_ci_low = ds_ev['WSE_ev_ci'].sel(alpha=alpha).squeeze() #.sel(ensemble='cnrs')

# diff and overlap ci
diff_cmpnd_surge = (da_wse_ev.sel(scen='surge') - da_wse_ev.sel(scen='tide'))
diff_cmpnd_surge.name = 'diff_surge'
diff_cmpnd_surge_sign = np.logical_or(
    da_wse_ev_ci_low.sel(scen='surge') > da_wse_ev_ci_high.sel(scen='tide'), # cmpnd > surge
    da_wse_ev_ci_high.sel(scen='surge') < da_wse_ev_ci_low.sel(scen='tide') # surge >  cpmnd
)
diff_cmpnd_surge_sign.name = 'diff_surge'

diff_cmpnd_runoff = (da_wse_ev.sel(scen='surge') - da_wse_ev.sel(scen='seas'))
diff_cmpnd_runoff.name = 'diff_seas'
diff_cmpnd_runoff_sign = np.logical_or(
    da_wse_ev_ci_low.sel(scen='cmpnd') > da_wse_ev_ci_high.sel(scen='runoff'),
    da_wse_ev_ci_high.sel(scen='cmpnd') < da_wse_ev_ci_low.sel(scen='runoff')
)
diff_cmpnd_runoff_sign.name = 'diff_runoff'

# main driver
cmpnd_l_runoff = da_wse_ev.sel(scen='cmpnd') > da_wse_ev.sel(scen='runoff')
cmpnd_l_surge = da_wse_ev.sel(scen='cmpnd') > da_wse_ev.sel(scen='surge')
runoff_le_surge = da_wse_ev.sel(scen='runoff') >= da_wse_ev.sel(scen='surge')
surge_l_runoff = da_wse_ev.sel(scen='surge') > da_wse_ev.sel(scen='runoff')
runoff_is_main_driver = np.logical_or(
    np.logical_or(
        np.logical_and(cmpnd_l_runoff, runoff_le_surge),
        np.logical_and(cmpnd_l_runoff==False, cmpnd_l_surge),
    ),
    np.logical_and(surge_l_runoff, cmpnd_l_runoff==False),
)
runoff_is_main_driver.name = 'runoff'
surge_is_main_driver = np.logical_or(
    np.logical_or(
        np.logical_and(cmpnd_l_surge, surge_l_runoff),
        np.logical_and(cmpnd_l_surge==False, cmpnd_l_runoff),
    ),
    np.logical_and(runoff_le_surge, cmpnd_l_surge==False),
)
surge_is_main_driver.name = 'surge'

ds_stats_ci = xr.merge([
    diff_cmpnd_runoff_sign,
    diff_cmpnd_surge_sign,
])
ds_stats = xr.merge([
    diff_cmpnd_runoff,
    diff_cmpnd_surge,
    runoff_is_main_driver,
    surge_is_main_driver,
]).compute()
ds_stats_ci

In [ ]:
# fn_ev_stats = join(ddir, 'rivmth_AMpeaks_d14_ev_gumb_stats.nc')
# ds_stats = xr.open_dataset(fn_ev_stats)

dim = 'ensemble'
ds_diff = ds_stats[['diff_runoff', 'diff_surge']]
# average and calculate significance based on std error
N = ds_diff[dim].size
ds_diff_mean = ds_diff.mean(dim)
ds_diff_dir =  xr.ufuncs.fabs(xr.ufuncs.sign(ds_diff).sum(dim)) >= N
ds_diff_sign_nonzero = xr.ufuncs.fabs(ds_diff_mean / ds_diff.std(dim)) > (2 / xr.ufuncs.sqrt(N-1))
ds_diff_sign_ci = ds_stats_ci[['diff_runoff', 'diff_surge']].sum(dim) >= N/2
ds_diff_sign = np.logical_and(ds_diff_sign_ci, ds_diff_dir)

ds_diff_stats = ds_diff_mean
ds_diff_stats['runoff'] = np.logical_and(
#     ds_stats['runoff'].sum(dim) >= ds_stats['surge'].sum(dim), 
    True,
    ds_diff_sign['diff_runoff']
)
ds_diff_stats['surge'] = np.logical_and(
#     ds_stats['runoff'].sum(dim) < ds_stats['surge'].sum(dim), 
    True,
    ds_diff_sign['diff_surge']
)
ds_diff_stats['compound'] = np.logical_or(ds_diff_stats['runoff'], ds_diff_stats['surge'])
ds_diff_stats['runoff'].sel(rp=10).sum(), ds_diff_stats['surge'].sel(rp=10).sum()

In [ ]:
rp=10
gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize']],
    ds_diff_stats.sel(rp=rp).reset_coords(drop=True).to_dataframe()
], axis=1))
gdf0 = gdf[gdf['compound']==False].copy()
gdf_surge = gdf[gdf['surge']].sort_values(by='diff_surge').copy()
gdf_runoff = gdf[gdf['runoff']].sort_values(by='diff_runoff').copy()
(gdf_runoff['diff_runoff'] < 0).sum() + (gdf_runoff['diff_surge'] < 0).sum()
gdf.loc[3975]

In [ ]:
label=f'change in riverine water level \n{rp}-yr return period [m]'
vmin, vmax, n= 0, 1, 6
# vmin, vmax, n= -0.1, 0.25, 8
cticks=np.linspace(vmin, vmax, n)
cmap_turbo_div = ListedColormap([
    interpolate_cmap(google_turbo_data, x) for x in 
    np.hstack([np.linspace(0.4-(.4*abs(vmin)/vmax)*2, .4, int(-vmin*100)), np.linspace(0.55, 1, int(vmax*100))])
])
# cmap = plt.cm.Greens
cmap = cmap_turbo_div
cmap.set_under('grey')

plt.close('all')
fig = plt.figure(figsize=(17, 12))
grid = plt.GridSpec(2, 6, hspace=0.0, wspace=0.25)

driver = 'surge'
name = 'runoff' if driver == 'surge' else 'surge'
column = f'diff_{name}'
gdf0 = gdf[gdf[name]==False].copy()
gdf1 = gdf[gdf[name]].sort_values(by=column).copy()

# main map
axg = fig.add_subplot(grid[:-1, :], projection=crs_sub)
add_regions(axg, regions, dx=-6, dy=-6)
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
# gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.8), markersize=5, linewidth=0.2, legend=False)
# gdf_runoff.plot(ax=axg, marker='o', color='green', markersize=15, linewidth=0.2, legend=False)
# gdf_surge.plot(ax=axg, marker='^', color='blue', markersize=15, linewidth=0.2, legend=False)
# plot_kwargs.update(markersize=gdf_surge['markersize'].values, marker='^')
# plot_choropleth(
#     fig, axg, gdf_surge, column='diff_surge', 
#     cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
#     plot_kwargs=plot_kwargs, cbar_kwargs=False,
# )
plot_kwargs.update(markersize=gdf1['markersize'].values, marker='o')
cax = plot_choropleth(
    fig, axg, gdf1, column=column, 
    cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
    plot_kwargs=plot_kwargs,
    cbar_kwargs=dict(label=label, location='right', extend='both'),
    cbar_pos = dict(pad=0.05, fraction=0.1)
)

# submaps 
axs = dict()
# gdf1 = gdf1.sort_values('markersize', ascending=True)
for i, (name, bbox) in enumerate(regions.items()):
    ax = fig.add_subplot(grid[-1, i*2:(i+1)*2], projection=crs_sub)
    basemap(ax, bbox=bbox, gridlines=True, gridspacing=10, outline=True, **bmap_kwargs)
    gdf0.plot(ax=ax, marker='x', color=(0, 0, 0, 0.8), markersize=15, linewidth=0.2, legend=False)
#     gdf_runoff.plot(ax=ax, marker='o', color='green', markersize=15, linewidth=0.2, legend=False)
#     gdf_surge.plot(ax=ax, marker='^', color='blue', markersize=15, linewidth=0.2, legend=False)
#     plot_kwargs.update(markersize=gdf_surge['markersize'].values*2, marker='^')
#     plot_choropleth(
#         fig, ax, gdf_surge, column='diff_surge', 
#         cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
#         plot_kwargs=plot_kwargs, cbar_kwargs=False,
#     )
    plot_kwargs.update(markersize=gdf1['markersize'].values*2, marker='o')
    cax = plot_choropleth(
        fig, ax, gdf1, column=column, 
        cmap=cmap, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
        plot_kwargs=plot_kwargs, cbar_kwargs=False,
    )
    xmin, ymax = bbox[0], bbox[3]
    ax.text(xmin, ymax+1, name, transform=crs_sub)
    axs[name] = ax

fn_fig = join(fdir, f'diff_{driver}_ev_gumb_rp{rp:02d}.png')
# fn_fig = join(fdir, f'surge_vs_runoff_rp{rp:02d}.png')
print(basename(fn_fig))
plt.savefig(fn_fig)

In [ ]:
from scipy.stats import pearsonr, zscore
import pandas as pd
def nanzscore(x):
    z = np.ones_like(x)
    valid = np.isnan(x)==False
    z[valid] = zscore(x[valid])
    return z 

## timing

In [ ]:
q = 95
min_dist = 30

fn_peaks_timing = join(ddir, f'rivmth_peaks_timing.nc')
ds_timing = xr.open_dataset(fn_peaks_timing)
ds_timing_diff = ds_timing['doy_diff']


dim = 'ensemble'
# average and calculate significance based on std error
N = ds_timing_diff[dim].size
ds_timing_diff_mean = ds_timing_diff.mean(dim)
ds_timing_diff_dir =  xr.ufuncs.fabs(xr.ufuncs.sign(ds_timing_diff).sum(dim)) == N
ds_timing_diff_sign = xr.ufuncs.fabs(ds_timing_diff_mean / ds_timing_diff.std(dim)) > (2 / xr.ufuncs.sqrt(N-1))
ds_timing_sign = (ds_timing['doy_uniform_p'].max(dim).max('scen')<0.05)

ds_timing_diff_stats = xr.concat([
    ds_timing_diff_mean,
    np.logical_and(np.logical_and(ds_timing_diff_sign, ds_timing_diff_dir), ds_timing_sign),
], dim='stat')
ds_timing_diff_stats['stat'] = xr.Variable('stat', ['mean', 'sign'])

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../3-postprocess/'))

from circ_stats import circ_mean
from temp_stats import doy_to_circ, circ_to_doy

circ = doy_to_circ(ds_timing['doy_mean'], dim=None)
ds_timing_mean = circ_to_doy(circ_mean(circ, dim='ensemble'), None)
ds_timing_sign = ds_timing['doy_uniform_p'].max(dim)<0.05

ds_timing_stats = xr.concat([
    ds_timing_mean,
    ds_timing_sign,
], dim='stat')
ds_timing_stats['stat'] = xr.Variable('stat', ['mean', 'sign'])
ds_timing_stats

In [ ]:
vmin, vmax, n= -50, 50, 5
label='change in mean flood day [days]'
cticks=np.linspace(vmin, vmax, n)

gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize']],
    ds_timing_diff_stats.to_series().unstack().T
], axis=1))

plt.close('all')
fig = plt.figure(figsize=(17, 10))
grid = plt.GridSpec(2, 6, hspace=0.05, wspace=0.4)

column = 'mean'
idx_sign = gdf['sign']==1

gdf0 = gdf[idx_sign==False].copy()
gdf1 = gdf[idx_sign].copy().sort_values('markersize', ascending=True)
df_box = ds_timing_diff.sel(index=gdf1.index).to_series().unstack(-1).T


# main map
axg = fig.add_subplot(grid[:-1, :-1], projection=crs_sub)
add_regions(axg, regions, dx=-6, dy=-6)
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.6), markersize=5, linewidth=0.2, alpha=0.6, legend=False)
plot_kwargs.update(markersize=gdf1['markersize'].values, marker='o', linewidth=0.8)
cax = plot_choropleth(
    fig, axg, gdf1, column=column, 
    cmap=plt.cm.RdBu_r, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
    plot_kwargs=plot_kwargs,
    cbar_kwargs=dict(label=label, location='right', extend='both'),
    cbar_pos = dict(pad=0.1, fraction=0.1)
)

# submaps 
axs = dict()
# gdf1 = gdf1.sort_values('markersize', ascending=True)
for i, (name, bbox) in enumerate(regions.items()):
    ax = fig.add_subplot(grid[-1, i*2:(i+1)*2], projection=crs_sub)
    basemap(ax, bbox=bbox, gridlines=True, gridspacing=10, outline=True, **bmap_kwargs)
    gdf0.plot(ax=ax, marker='x', color=(0, 0, 0, 0.6), markersize=15, linewidth=0.2, legend=False)
    plot_kwargs.update(markersize=gdf1['markersize'].values*2, marker='o')
    plot_choropleth(
        fig, ax, gdf1, column=column, 
        cmap=plt.cm.RdBu_r, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
        plot_kwargs=plot_kwargs, cbar_kwargs=False
    )
    xmin, ymax = bbox[0], bbox[3]
    ax.text(xmin, ymax+1, name, transform=crs_sub)
    axs[name] = ax

# boxplot ensemble
color_props = dict(boxes="lightgrey", whiskers="k", medians="k", caps="k")
ax = fig.add_subplot(grid[:-1, -1:])
df_box.plot.box(ax=ax, color=color_props, patch_artist=True, **box_kwargs)
ax.set_ylim([vmin, vmax])
ax.yaxis.set_ticks(cticks)
ax.yaxis.set_ticklabels([])
ax.grid(axis='x')
ax.set_xlabel('model')
cpos = cax.get_position()
bpos = ax.get_position()
cax.set_position([cpos.x0, cpos.y0, cpos.width, cpos.height])
# ax.set_position([cpos.x0+0.03, cpos.y0, bpos.width*1.5, cpos.height])
ax.set_position([cpos.x0+0.03, cpos.y0+cpos.height*0.05, bpos.width*1.5, cpos.height/1.10])
cax.yaxis.set_ticks_position('left')
cax.yaxis.set_label_position('left')

fn_fig = join(fdir, f'timing_q{q}d{min_dist}_doy_diff.png')
print(basename(fn_fig))
plt.savefig(fn_fig)
# plt.close('all')


In [ ]:
amin, amax = -np.pi, np.pi
norm = mpl.colors.Normalize(-np.pi, np.pi)

# legend
dates = pd.date_range(date(2001,1,1), date(2001,12,31), freq='MS')[0::2]
dticks = dates.strftime('%b').tolist()
doys = np.array([d.timetuple().tm_yday for d in dates]).astype(float)
circ = doys/365*2*np.pi # we need to use a [0, 2pi] range here
xval = np.arange(amin, amax, 0.1)
yval = np.ones_like(xval)*2.

In [ ]:
plt.close('all')
fig = plt.figure(figsize=(17, 12))
grid = plt.GridSpec(2, 2, hspace=0.0, wspace=0.1)

# main map chnage
vmin, vmax, n= -50, 50, 5
label='change in mean annual flood day [days]'
cticks=np.linspace(vmin, vmax, n)
gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize']],
    ds_timing_diff_stats.to_series().unstack().T
], axis=1))
column = 'mean'
idx_sign = gdf['sign']==1
gdf0 = gdf[idx_sign==False].copy()
gdf1 = gdf[idx_sign].copy().sort_values('markersize', ascending=True)
axg = fig.add_subplot(grid[0, :], projection=crs_sub)
# add_regions(axg, regions, dx=-6, dy=-6)
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.6), markersize=5, linewidth=0.2, alpha=0.6, legend=False)
plot_kwargs.update(markersize=gdf1['markersize'].values, marker='o', linewidth=0.8)
cax = plot_choropleth(
    fig, axg, gdf1, column=column, 
    cmap=plt.cm.RdBu_r, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
    plot_kwargs=plot_kwargs,
    cbar_kwargs=dict(label=label, location='right', extend='both'),
    cbar_pos = dict(pad=0.05, fraction=0.1)
)
axg.text(-175, 80, letters[0], transform=crs_sub, fontsize=16, weight='bold')

# tide
axg1 = fig.add_subplot(grid[1,0], projection=crs_sub)
gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize']],
    ds_timing_stats.sel(scen='tide').to_series().unstack().T
], axis=1))
circ_doy = gdf['mean']/365*2*np.pi
gdf['color'] = np.where(circ_doy>np.pi, -2*np.pi+circ_doy, circ_doy)
gdf0 = gdf[idx_sign==False].copy()
gdf1 = gdf[idx_sign].copy().sort_values('markersize', ascending=True)
# main map
basemap(axg1, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
gdf0.plot(ax=axg1, marker='x', color=(0, 0, 0, 0.6), markersize=5, linewidth=0.2, alpha=0.6, legend=False)
gdf1.plot(ax=axg1, column='color', markersize=20, 
          norm=norm, vmin=amin, vmax=amax, cmap=cmap_phase,
          edgecolor=(0.5, 0.5, 0.5, 0.1), linewidth=0, legend=False)
axg1.text(-175, 75, letters[1], transform=crs_sub, fontsize=16, weight='bold')

# surge
axg2 = fig.add_subplot(grid[1,1], projection=crs_sub)
gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize']],
    ds_timing_stats.sel(scen='surge').to_series().unstack().T
], axis=1))
circ_doy = gdf['mean']/365*2*np.pi
gdf['color'] = np.where(circ_doy>np.pi, -2*np.pi+circ_doy, circ_doy)
gdf0 = gdf[idx_sign==False].copy()
gdf1 = gdf[idx_sign].copy().sort_values('markersize', ascending=True)

# main map
basemap(axg2, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
gdf0.plot(ax=axg2, marker='x', color=(0, 0, 0, 0.6), markersize=5, linewidth=0.2, alpha=0.6, legend=False)
gdf1.plot(ax=axg2, column='color', markersize=20, 
          norm=norm, vmin=amin, vmax=amax, cmap=cmap_phase,
          edgecolor=(0.5, 0.5, 0.5, 0.1), linewidth=0, legend=False)
axg2.text(-175, 75, letters[2], transform=crs_sub, fontsize=16, weight='bold')

# add colorbar
posn = axg1.get_position() # fix colorbar position
cbar_ax = fig.add_axes([0, 0, 0.1, 0.1], polar=True) # new ax at random location.
cbar_ax.set_theta_offset(0.5*np.pi)
cbar_ax.set_theta_direction(-1)
cbar_ax.scatter(xval, yval, c=xval, s=45, cmap=cmap_phase, norm=norm, linewidths=0)
cbar_ax.set_yticks([])
cbar_ax.set_xticks(circ)
cbar_ax.set_xticklabels(dticks)
size = posn.height/4.
cbar_ax.set_position([posn.x0+size/4., posn.y0+posn.height/2-size*1.5, size, size])


fn_fig = join(fdir, f'timing_q{q}d{min_dist}_doy_mean.png')
print(basename(fn_fig))
plt.savefig(fn_fig)
# plt.close('all')